In [1]:
import h5py
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_row',10000)

In [25]:
useful_strings = [
    "PhotonEnergy",
    "PolarMode",
    "SampleTemp",
    "SampleXs",
    "SampleYs",
    "SampleZ",
    "SampleTheta",
    "SamplePhi",
    "SampleTilt",
    "AcquireTime",
    "ExposureSplit",
    "ExitSlit",
    "BeamCurrent",
    ]
try:
    RIXSinfo
except NameError:
    RIXSinfo = pd.DataFrame(columns = ['FileName'] + useful_strings)

In [26]:
def polar_trans(PolarMode):                           
    if PolarMode == 0:
        Polarization = "LH"
    elif PolarMode == 1:
        Polarization = "LV"
    elif PolarMode == 2:
        Polarization = "C+"
    else:
        Polarization = "C-"
    return Polarization

def load_meta(file_name,path):
    global useful_strings  
    f = h5py.File(f"{path}/{file_name}_d1.h5",'r')
    meta_data = {}
    NDAttributes = f['entry']['instrument']['NDAttributes']
    meta_data['FileName'] = file_name
    for key in useful_strings:
        meta_data[key] = round(np.mean(NDAttributes[key]),3)   
    meta_data["PolarMode"] = polar_trans(meta_data["PolarMode"])      
    f.close()
    return meta_data

In [57]:
def scan_information(file_list,path):
    global RIXSinfo
    for filename in file_list:
        file_name = filename[:-6]
        if file_name not in RIXSinfo["FileName"].values:
            try:
                meta_data = pd.DataFrame([load_meta(file_name,path)]) 
                RIXSinfo = pd.concat([RIXSinfo,meta_data],ignore_index = True)
            except:
                pass
    RIXSinfo.sort_values("FileName",inplace = True,ignore_index = True)
    return RIXSinfo

In [28]:
def listFile(fileDir):
    list = sorted(os.listdir(fileDir))
    # list.sort(key=lambda fn: os.path.getmtime(fileDir + fn))
    return list

In [29]:
import ipywidgets as widgets

path = widgets.Text(
    value='',
    placeholder='X://RIXS/Asmara',
    description='Path:',
    disabled=False
)

# display(path)

scan = widgets.Button(description="Scan")
output = widgets.Output()

display(path,scan, output)

Text(value='', description='Path:', placeholder='X://RIXS/Asmara')

Button(description='Scan', style=ButtonStyle())

Output()

In [34]:
def do_scan():
    global RIXSinfo
    try:
        list = listFile(path.value)
        RIXSinfo = scan_information(list[::3],path.value)
    except:
        pass


def on_button_clicked(b):
    output.clear_output()

    with output:
        do_scan()
        display(RIXSinfo)

scan.on_click(on_button_clicked)

In [8]:
# RIXSinfo.to_csv(f"{path}/logbook.csv",index=False) 